In [ ]:
import os
import pandas as pd
import numpy as np
import openmatrix as omx
import random
import yaml
from pathlib import Path
from utility import *

import warnings
warnings.filterwarnings('ignore')

import pandas as pd
pd.options.display.float_format = '{:,.3f}'.format

In [ ]:
with open('config.yaml', 'r') as file:
    params = yaml.safe_load(file)
    
_join = os.path.join
_dir = os.path.dirname
_norm = os.path.normpath

# paths
ctramp_dir = params['ctramp_dir']
model_outputs_dir = params['model_dir']
summary_dir = params['summary_dir']
concept_id = params['concept_id']
preprocess_dir = _join(ctramp_dir, '_pre_process_files')
perf_measure_columns = params['final_columns']
model_year = params['model_year']
filename_extension = params['filename_extension']
hwy_skims_dir = _join(model_outputs_dir, r'skims\highway' )

skims_dir = _join(model_outputs_dir, r'skims')

In [ ]:
#od_pairs = [(1954-1,64-1),  (1954-1, 1969-1), (1954-1,2153-1), (2153-1,2484-1)]
#od_pairs = [(1517-1,1046-1), (1438-1, 1433-1), (1644-1, 1655-1), (1433-1,1438-1), (1310-1, 1309-1)]
od_pairs = [(66,64), (79,84), (60,24), (2563, 2277), (2277, 697), (1903,2226)]

In [ ]:
%%time
# The data tab of the UEC file lists all the matrix cores and location an matrix files of skims
# 1 for EA, 2 for AM, 3 for MD, 4 for PM and 5 for EV

df_temp = []

# extract the file names, matrix cores 
matrix_df = pd.read_excel(_join(params['common_dir'], r"TripModeChoice.xlsx"), sheet_name='data')
matrix_df = matrix_df.iloc[9:]
matrix_df.columns = ['no', 'token', 'format', 'file','matrix', 'group', 'index']
#matrix_df[1:5]

# pre-processing
matrix_df['matrix_files'] = matrix_df['file'].str.replace('skims/', '')
matrix_df['path'] = 'skims'
matrix_df.loc[matrix_df['matrix_files'].str.contains('nonmot')==True, 'path'] = 'active'
matrix_df.loc[matrix_df['matrix_files'].str.contains('trnskm')==True, 'path'] = 'transit'
matrix_df.loc[matrix_df['matrix_files'].str.contains('hwyskm')==True, 'path'] = 'highway'
#matrix_df[1:5]

# Iterate over the DataFrame rows
for _, row in matrix_df.iterrows():
    variable_name = row['token']
    file_path = row['path']
    filename = row['matrix_files']
    matrix_cr = row['matrix']
    
    # Extract the variable name and index (if present)
    if '[' in variable_name:
        name_start = variable_name.index('[')
        name_end = variable_name.index(']')
        index = int(variable_name[name_start+1:name_end])
        variable_name = variable_name[:name_start]
    else:
        index=None
    
    # Read the file using numpy.load() and assign it to the variable with the specified index
    file = omx.open_file(_join(skims_dir, file_path, filename))
    file_contents = np.array(file[matrix_cr])
    print(filename)

    values_to_store = []
    for index in od_pairs:
        extracted_values = file_contents[index[0]-1, index[1]-1] 
        values_to_store.append(extracted_values)
        
    #values_to_store = np.array(values_to_store).T.tolist()
    df = pd.DataFrame([values_to_store], columns=['od1', 'od2', 'od3', 'od4', 'od5', 'od6']) #Update this 
    df['matrix_core'] = filename
    df['token'] = row['token']
    df['group'] = row['group']
    df['matrix'] = row['matrix']
    
    df_temp.append(df)
    #break

In [ ]:
df_skims = pd.concat(df_temp)

In [ ]:
df_skims.to_csv("df_skims_r39.csv")